In [6]:
import sys
from pathlib import Path
import os

# Add project root to Python path (notebook is in notebooks/ folder, project root is one level up)
notebook_dir = Path(os.getcwd())
project_root = notebook_dir.parent if notebook_dir.name == 'notebooks' else notebook_dir
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import pandas as pd
from src.models_garch import garch_model, garch_model_forecast

In [8]:
# Load returns data
# Get project root path (same logic as in first cell)
notebook_dir = Path(os.getcwd())
project_root = notebook_dir.parent if notebook_dir.name == 'notebooks' else notebook_dir
csv_path = project_root / 'SPY_returns.csv'

df = pd.read_csv(csv_path, parse_dates=["Date"], index_col="Date")

In [12]:
# Fit the GARCH model
res, garch_vol = garch_model(df['Returns'], 1, 1)


C:\Users\TNath\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\arch\univariate\base.py:694: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0001181. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  self._check_scale(resids)


ValueError: not enough values to unpack (expected 2, got 1)

In [13]:
# Add to DataFrame
df["GARCH_vol"] = garch_vol

# Forecast volatility
var_forecast = garch_model_forecast(df['Returns'], 1, 1)

print("5-day forecasted volatility: ", var_forecast)


NameError: name 'garch_vol' is not defined